In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from models.HybridGNet2IGSC import Hybrid 

import numpy as np
import torch

from utils.utils import scipy_to_torch_sparse, genMatrixesLungs, genMatrixesLungsHeart, genMatrixesLHC
import scipy.sparse as sp
from utils.dataset_full import LandmarksDataset, ToTensor

In [5]:
test_lungs = open("test_images_lungs.txt",'r').read().splitlines()
test_heart = open("test_images_heart.txt",'r').read().splitlines()

test_mont = [image for image in test_lungs if "MCU" in image]
test_shen = [image for image in test_lungs if "CHN" in image]
test_jsrt = [image for image in test_heart if "JP" in image]
test_pad = [image for image in test_heart if not "JP" in image]

dataset_mont = LandmarksDataset(test_mont,
                           img_path = "../Chest-xray-landmark-dataset/Images",
                           label_path = "../Chest-xray-landmark-dataset/landmarks",
                           organ = 'L',
                           transform = ToTensor()
                           )

dataset_shen = LandmarksDataset(test_shen,
                           img_path = "../Chest-xray-landmark-dataset/Images",
                           label_path = "../Chest-xray-landmark-dataset/landmarks",
                           organ = 'L',
                           transform = ToTensor()
                           )


dataset_jsrt = LandmarksDataset(test_jsrt,
                           img_path = "../Chest-xray-landmark-dataset/Images",
                           label_path = "../Chest-xray-landmark-dataset/landmarks",
                           organ = 'LHC',
                           transform = ToTensor()
                           )

dataset_pad = LandmarksDataset(test_pad,
                           img_path = "../Chest-xray-landmark-dataset/Images",
                           label_path = "../Chest-xray-landmark-dataset/landmarks",
                           organ = 'LH',
                           transform = ToTensor()
                           )

device = 'cuda:0'

In [6]:
A, AD, D, U = genMatrixesLHC()

N1 = A.shape[0]
N2 = AD.shape[0]

A = sp.csc_matrix(A).tocoo()
AD = sp.csc_matrix(AD).tocoo()
D = sp.csc_matrix(D).tocoo()
U = sp.csc_matrix(U).tocoo()

D_ = [D.copy()]
U_ = [U.copy()]

config = {}

config['n_nodes'] = [N1, N1, N1, N2, N2, N2]
A_ = [A.copy(), A.copy(), A.copy(), AD.copy(), AD.copy(), AD.copy()]
A_t, D_t, U_t = ([scipy_to_torch_sparse(x).to(device) for x in X] for X in (A_, D_, U_))

config['latents'] = 64
config['inputsize'] = 1024

f = 32
config['filters'] = [2, f, f, f, f//2, f//2, f//2]
config['skip_features'] = f

hybrid_jsrt = Hybrid(config.copy(), D_t, U_t, A_t).to(device)
hybrid_jsrt.load_state_dict(torch.load("weights/HybridGNet/Hybrid_LHC_STRICT/bestMSE.pt"))
hybrid_jsrt.eval()

hybrid_multi_cla = Hybrid(config.copy(), D_t, U_t, A_t).to(device)
hybrid_multi_cla.load_state_dict(torch.load("weights/HybridGNet/Hybrid_LHC_FULL/bestMSE.pt"))
hybrid_multi_cla.eval()

print('Model loaded')

Model loaded


In [7]:
A, AD, D, U = genMatrixesLungsHeart()

N1 = A.shape[0]
N2 = AD.shape[0]

A = sp.csc_matrix(A).tocoo()
AD = sp.csc_matrix(AD).tocoo()
D = sp.csc_matrix(D).tocoo()
U = sp.csc_matrix(U).tocoo()

D_ = [D.copy()]
U_ = [U.copy()]

config = {}

config['n_nodes'] = [N1, N1, N1, N2, N2, N2]
A_ = [A.copy(), A.copy(), A.copy(), AD.copy(), AD.copy(), AD.copy()]
A_t, D_t, U_t = ([scipy_to_torch_sparse(x).to(device) for x in X] for X in (A_, D_, U_))

config['latents'] = 64
config['inputsize'] = 1024

f = 32
config['filters'] = [2, f, f, f, f//2, f//2, f//2]
config['skip_features'] = f

hybrid_heart = Hybrid(config.copy(), D_t, U_t, A_t).to(device)
hybrid_heart.load_state_dict(torch.load("weights/HybridGNet/Hybrid_LH_STRICT/bestMSE.pt"))
hybrid_heart.eval()

hybrid_multi = Hybrid(config.copy(), D_t, U_t, A_t).to(device)
hybrid_multi.load_state_dict(torch.load("weights/HybridGNet/Hybrid_LH_FULL/bestMSE.pt"))
hybrid_multi.eval()

print('Model loaded')

Model loaded


In [8]:
A, AD, D, U = genMatrixesLungs()

N1 = A.shape[0]
N2 = AD.shape[0]

A = sp.csc_matrix(A).tocoo()
AD = sp.csc_matrix(AD).tocoo()
D = sp.csc_matrix(D).tocoo()
U = sp.csc_matrix(U).tocoo()

D_ = [D.copy()]
U_ = [U.copy()]

config = {}

config['n_nodes'] = [N1, N1, N1, N2, N2, N2]
A_ = [A.copy(), A.copy(), A.copy(), AD.copy(), AD.copy(), AD.copy()]
A_t, D_t, U_t = ([scipy_to_torch_sparse(x).to('cuda:0') for x in X] for X in (A_, D_, U_))

config['latents'] = 64
config['inputsize'] = 1024

f = 32
config['filters'] = [2, f, f, f, f//2, f//2, f//2]
config['skip_features'] = f

hybrid_lungs = Hybrid(config.copy(), D_t, U_t, A_t).to(device)
hybrid_lungs.load_state_dict(torch.load("weights/HybridGNet/Hybrid_L_BOTH/bestMSE.pt"))
hybrid_lungs.eval()

print('Model loaded')

Model loaded


In [9]:
from medpy.metric import dc, hd
import cv2

def getDenseMask(RL, LL, H = None, CLA1 = None, CLA2 = None, imagesize = 1024):
    img = np.zeros([1024,1024])
    imgcla = np.zeros([1024,1024])
    
    RL = RL.reshape(-1, 1, 2).astype('int')
    LL = LL.reshape(-1, 1, 2).astype('int')

    img = cv2.drawContours(img, [RL], -1, 1, -1)
    img = cv2.drawContours(img, [LL], -1, 1, -1)
    
    if H is not None:
        H = H.reshape(-1, 1, 2).astype('int')
        img = cv2.drawContours(img, [H], -1, 2, -1)
        
    if CLA1 is not None:
        CLA1 = CLA1.reshape(-1, 1, 2).astype('int')
        imgcla = cv2.drawContours(imgcla, [CLA1], -1, 3, -1)
    
    if CLA2 is not None:
        CLA2 = CLA2.reshape(-1, 1, 2).astype('int')
        imgcla = cv2.drawContours(imgcla, [CLA2], -1, 3, -1)
    
    return img, imgcla

def getDenseMask2(RL, LL, H = None, CLA1 = None, CLA2 = None, imagesize = 1024):
    img = np.zeros([1024,1024])
    
    RL = RL.reshape(-1, 1, 2).astype('int')
    LL = LL.reshape(-1, 1, 2).astype('int')

    img = cv2.drawContours(img, [RL], -1, 1, -1)
    img = cv2.drawContours(img, [LL], -1, 1, -1)
    
    if H is not None:
        H = H.reshape(-1, 1, 2).astype('int')
        img = cv2.drawContours(img, [H], -1, 2, -1)
        
    if CLA1 is not None:
        CLA1 = CLA1.reshape(-1, 1, 2).astype('int')
        img = cv2.drawContours(img, [CLA1], -1, 3, -1)
    
    if CLA2 is not None:
        CLA2 = CLA2.reshape(-1, 1, 2).astype('int')
        img = cv2.drawContours(img, [CLA2], -1, 3, -1)
    
    return img

def evalImageMetricsCla(output, target, output_cla, target_cla):
    dcp = dc(output == 1, target == 1)
    dcc = dc(output == 2, target == 2)
    dcla = dc(output_cla == 3, target_cla == 3)
    
    hdp = hd(output == 1, target == 1)
    hdc = hd(output == 2, target == 2)
    hdcla = hd(output_cla == 3, target_cla == 3)
        
    return [dcp, dcc, dcla, hdp, hdc, hdcla]

def evalImageMetrics(output, target):
    dcp = dc(output == 1, target == 1)
    dcc = dc(output == 2, target == 2)
    
    hdp = hd(output == 1, target == 1)
    hdc = hd(output == 2, target == 2)
        
    return [dcp, dcc, -1, hdp, hdc, -1]

def evalImageMetricsLungs(output, target):
    dcp = dc(output == 1, target == 1)
    hdp = hd(output == 1, target == 1)
        
    return [dcp, -1, -1, hdp, -1, -1]

In [10]:
import pandas as pd
from sklearn.metrics import mean_squared_error

model_list = [hybrid_lungs, hybrid_heart, hybrid_multi, hybrid_jsrt, hybrid_multi_cla]
model_names = ['L (Full)', 'LH (Strict)', 'LH (Full)', 'LHC (Strict)', 'LHC (Full)']

results = pd.DataFrame()

datasets = [dataset_mont,dataset_shen,dataset_jsrt, dataset_pad]
dataset_names = ["Montgomery (L)", "Shenzhen (L)", "JSRT (LHC)", "Padchest (LH)"]

In [11]:
for k in range(0,4):
    dataset = datasets[k]

    for i in range(0, len(dataset.images)):
        print('\r',dataset_names[k], i+1,'of', len(dataset.images),end='')
        with torch.no_grad():
            sample = dataset[i]

            data, target = sample['image'], sample['landmarks']
            data = torch.unsqueeze(data, 0).to(device)
            target =  target.reshape(-1,2).numpy()
            
            RL = target[:44] * 1024
            LL = target[44:94] * 1024

            if k > 1:
                H = target[94:120] * 1024
            else:
                H = None
            
            if k == 2:
                CLA1 = target[120:143] * 1024
                CLA2 = target[143:] * 1024
            else:
                CLA1 = None
                CLA2 = None

            targetseg, targetcla = getDenseMask(RL, LL, H, CLA1, CLA2)

            for j in range(0, len(model_list)):
                output = model_list[j](data)
                
                if len(output) > 1:
                    output = output[0]
                    
                output = output.cpu().numpy().reshape(-1, 2) 

                mse_l = mean_squared_error(output[:94,:].reshape(-1) * 1024, target[:94,:].reshape(-1) * 1024)
                
                RL = output[:44] * 1024
                LL = output[44:94] * 1024
                
                if k > 1 and j > 0:
                    H = output[94:120] * 1024
                    mse_h = mean_squared_error(output[94:120,:].reshape(-1) * 1024, target[94:120,:].reshape(-1) * 1024)
                else:
                    H = None
                    mse_h = -1
                
                if k == 2 and j > 2:
                    CLA1 = output[120:143] * 1024
                    CLA2 = output[143:] * 1024
                    mse_cla = mean_squared_error(output[120:,:].reshape(-1) * 1024, target[120:,:].reshape(-1) * 1024)
                else:
                    mse_cla = -1
                    CLA1 = None
                    CLA2 = None

                outseg, outsegcla = getDenseMask(RL, LL, H, CLA1, CLA2)

                if k == 2 and j > 2:
                    metrics = evalImageMetricsCla(outseg, targetseg, outsegcla, targetcla)
                elif k > 1 and j > 0:
                    metrics = evalImageMetrics(outseg, targetseg)
                else:
                    metrics = evalImageMetricsLungs(outseg, targetseg)

                aux = pd.DataFrame([[i, dataset_names[k], model_names[j], mse_l, mse_h, mse_cla] + metrics], 
                                     columns=['i', 'Dataset', 'Model', 'MSE Lungs', 'MSE Heart', 'MSE Cla', 
                                     'Dice Lungs','Dice Heart','Dice Cla','HD Lungs','HD Heart','HD Cla'])
                results = results.append(aux, ignore_index = True)
    print('')

 Montgomery (L) 27 of 27
 Shenzhen (L) 78 of 78
 JSRT (LHC) 49 of 49
 Padchest (LH) 27 of 27


In [12]:
for d in dataset_names:
    print(d)
    sub = results[results['Dataset'] == d]
    del sub['i']

    if '(L)' in d:
            sub = sub[['MSE Lungs', 'Dice Lungs', 'HD Lungs', 'Model']]
    elif '(LH)' in d:
        sub = sub[['MSE Lungs', 'Dice Lungs', 'HD Lungs', 'MSE Heart', 'Dice Heart', 'HD Heart', 'Model']]
    elif '(LHC)' in d:
        sub = sub[['MSE Lungs', 'Dice Lungs', 'HD Lungs', 'MSE Heart', 'Dice Heart', 'HD Heart', 'MSE Cla', 'Dice Cla', 'HD Cla', 'Model']]

    display(sub.groupby('Model').mean())

Montgomery (L)


,MSE Lungs,Dice Lungs,HD Lungs
Model,,,
L (Full),128.452675,0.969946,27.445437
LH (Full),104.431027,0.970544,27.825295
LH (Strict),295.543605,0.953532,38.497491
LHC (Full),110.704940,0.968461,26.214971
LHC (Strict),571.523590,0.937095,49.217751


Shenzhen (L)


,MSE Lungs,Dice Lungs,HD Lungs
Model,,,
L (Full),142.851287,0.964827,32.515606
LH (Full),145.943651,0.964642,32.809856
LH (Strict),264.425423,0.947935,43.055797
LHC (Full),148.740172,0.963006,33.186884
LHC (Strict),486.012668,0.934273,51.049109


JSRT (LHC)


,MSE Lungs,Dice Lungs,HD Lungs,MSE Heart,Dice Heart,HD Heart,MSE Cla,Dice Cla,HD Cla
Model,,,,,,,,,
L (Full),120.239756,0.972898,33.014314,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
LH (Full),124.846900,0.973544,31.011027,375.763711,0.937294,33.817791,-1.000000,-1.000000,-1.000000
LH (Strict),155.694765,0.970739,36.273058,382.426159,0.935626,34.529578,-1.000000,-1.000000,-1.000000
LHC (Full),122.162627,0.972690,31.047529,390.813508,0.936021,34.222853,101.120760,0.823596,22.932518
LHC (Strict),139.414511,0.970653,33.958884,413.042040,0.932717,35.708829,83.397552,0.840727,20.824964


Padchest (LH)


,MSE Lungs,Dice Lungs,HD Lungs,MSE Heart,Dice Heart,HD Heart
Model,,,,,,
L (Full),152.057760,0.957582,36.563907,-1.000000,-1.000000,-1.000000
LH (Full),187.033489,0.955762,38.829597,341.998322,0.935602,36.005012
LH (Strict),201.805065,0.952744,41.036891,351.686427,0.935155,36.812052
LHC (Full),172.015096,0.956503,37.136851,235.034342,0.944861,30.544074
LHC (Strict),480.207664,0.923092,56.834354,1318.033399,0.869064,70.905940
